## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

In [1]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np

In [2]:
nb = pd.read_csv('nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

#### Task I
Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg).

In [4]:
# done

In [3]:
x

,AST,REB,STL
0,41,43,14
1,23,43,8
2,20,39,7
3,19,47,6
4,21,43,4
...,...,...,...
7375,17,39,10
7376,26,40,10
7377,23,52,8
7378,23,41,11



#### TASK II
Create a function `psi(x)`, which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)

**Input:** DataFrame x from above. It contains columns AST, REB, STL

**Output:** DataFrame with columns: AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB. The number of rows should be the same as Input.

In [4]:
def psi(x):
    num_cols = range(len(x.columns))
    x2 = x.copy()
    for col in num_cols:
        x2[x.columns[col]+'^2']=x.iloc[:,col]**2
    for col in num_cols: 
        x2[x.columns[col]+x.columns[col-1]]=x.iloc[:,col]*x.iloc[:,col-1]
    x2['1']=1
    cols = x2.columns.tolist()
    cols = cols[0:len(x.columns)] + [cols[-1]] + cols[3:-1]
    x2 = x2[cols]
    return x2

In [5]:
len(x.columns)

3

In [6]:
x2 = x.copy()

In [7]:
x2 = psi(x2)

In [8]:
x2

,AST,REB,STL,1,AST^2,REB^2,STL^2,ASTSTL,REBAST,STLREB
0,41,43,14,1,1681,1849,196,574,1763,602
1,23,43,8,1,529,1849,64,184,989,344
2,20,39,7,1,400,1521,49,140,780,273
3,19,47,6,1,361,2209,36,114,893,282
4,21,43,4,1,441,1849,16,84,903,172
...,...,...,...,...,...,...,...,...,...,...
7375,17,39,10,1,289,1521,100,170,663,390
7376,26,40,10,1,676,1600,100,260,1040,400
7377,23,52,8,1,529,2704,64,184,1196,416
7378,23,41,11,1,529,1681,121,253,943,451


In [318]:
# cols = cols[0:3] + [cols[-1]] + cols[3:-1]
# print(cols)

[['AST', 'REB', 'STL']]


#### Task III
Create a transformed data matrix **X**, where each **x** is mapped to psi(x).

HINT: We need to apply our function from Task II to matrix (DataFrame) x

In [9]:
X = psi(x)

#### Task IV
Create a function `p2(x,w)`, which outputs the value of the polynomial ata given row of `x` for given parameters `w`.

Inputs:
- x: DataFrame from above
- w: vector which represents beta coeficients for each column of X from the Task III.

Ouputs:
- Series of the same length as DataFrame x. Each value is a dot product of particular row in DataFrame and coeficients `w`.


HINT:
- length of w needs to be the same as number of columns of the dataframe that is returned from function `psi(x)`

Example Input:

`p2(x, [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01])`

Example Output:

```
0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

```

Our columns in the DataFrame **X** that is the output of `psi(x)` were in this order: `AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB`. If your columns are in different order the result can be different even for the **w**.

In [10]:
np.random.seed(15)
w = np.random.rand(10)

In [11]:
w

array([0.8488177 , 0.17889592, 0.05436321, 0.36153845, 0.27540093,
       0.53000022, 0.30591892, 0.30447436, 0.11174128, 0.24989901])

In [37]:
def p2(x,w):
    X = psi(x)
    res_list=[]
    for indx,row in X.iterrows(): 
        res = np.dot(row, w)
        res_list.append(res)
    return pd.Series(res_list)

In [38]:
testp2 = p2(x,w)
print(testp2)

0       2068.703517
1       1425.748759
2       1153.983153
3       1511.393234
4       1301.874465
           ...     
7375    1161.930837
7376    1390.227413
7377    1921.632187
7378    1396.559223
7379    1281.715277
Length: 7380, dtype: float64


In [14]:
testp2

0       2068.703517
1       1425.748759
2       1153.983153
3       1511.393234
4       1301.874465
           ...     
7375    1161.930837
7376    1390.227413
7377    1921.632187
7378    1396.559223
7379    1281.715277
Length: 7380, dtype: float64

#### Task V
Create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**. We have specified **y** as the variable PTS above. We will predict scored points (PTS) based on assists, rebounds and steals.


HINTS: 
- Output of `p2(x,w)` represents our predictions. `y_pred = p2(x,w)`
- Loss can be computed as:

```
np.mean((y_pred - y)**2)
```

In [35]:
def Loss(x,y,w):
    y_pred = p2(x,w)
    loss = np.mean((y_pred - y)**2)
    return loss

In [36]:
Loss(x,y,w)

1936041.1827870472

#### Task VI
Code up the gradient descent. It should input **x**, target variable **y**, stepsize **alpha**, coeficients **w** and **maximum number of iterations**.

Steps:
1. transform input `x`
2. compute initial loss for given, x,y and w and append it to the empty list.
3. Inside the for loop, update each element of **w**, **w[i]**, using gradient descent.

HINT: `w[i] = w[i] - alpha * (1.0 / m) * errors_i.sum()` where `errors_i = (y_pred - y) * X.iloc[:, i]`. We are scaling the errors by multiplicating with values that are relevant for coeficients `w[i]` (column `i` of DataFrame `X`, output of `psi(x)`).

4. compute new loss using updated `w` and append to the list that we created in the step 2.
5. repeat steps 3 and 4 for max number of iterations times.

In [39]:
np.random.seed(15)
w = np.random.rand(10)
w

array([0.8488177 , 0.17889592, 0.05436321, 0.36153845, 0.27540093,
       0.53000022, 0.30591892, 0.30447436, 0.11174128, 0.24989901])

In [52]:
def gradient_descent(x,y,w,alpha=0.01,max_iter=10):
    X=psi(x)
    past_loss=[]
    past_loss.append(Loss(x,y,w))
    for i in range(max_iter):
        y_pred = p2(x,w)
        for j in range(len(w)):
            w[j] = w[j] - alpha * (1/y.size) * (np.dot((y_pred-y), X.iloc[:,j]))
        loss = Loss(x,y,w)
        past_loss.append(loss)
    return past_loss, w

#### Task VII
Choose an arbitrary **w** and **alpha** and run gradient descent for 100 iterations. How does the loss behave? Does it converge to something?


In [53]:
alpha = 0.01
max_iter=10 

In [54]:
gradient_descent(x,y,w,alpha,max_iter)

([2.385979618200766e+195,
  6.795791097083544e+204,
  1.9355897377709267e+214,
  5.512982343692057e+223,
  1.5702177857618566e+233,
  4.4723232200153555e+242,
  1.2738153373153833e+252,
  3.6281043067686236e+261,
  1.0333633514363935e+271,
  2.9432445315855005e+280,
  8.383003287920605e+289],
 array([3.73027416e+139, 7.45203179e+139, 1.28132256e+139, 1.65935972e+138,
        8.80700626e+140, 3.41885469e+141, 1.13582892e+140, 2.91089333e+140,
        1.67675217e+141, 5.71817562e+140]))

In [56]:
np.random.seed(15)
w = np.random.rand(10)
w

array([0.8488177 , 0.17889592, 0.05436321, 0.36153845, 0.27540093,
       0.53000022, 0.30591892, 0.30447436, 0.11174128, 0.24989901])

#### Task VIII
Can you find which **alpha** from `[1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]` has the smallest loss after 100 iterations for a given **w**?

In [59]:
alphas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
for a in alphas: 
    print(gradient_descent(x,y,w,a,max_iter=5)[0])

[inf, inf, inf, inf, inf, inf]
[inf, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan, nan]
